# 自然语言处理
----------------
## 一. 词嵌入-Word Embedding
#### 1. one-hot与featurized representation  
( 1 ) 用单词在Directionary中的位置做representation的方法, 即为one-hot. 这种表示方法, 无法代表给出词与词之间更多的信息.例如"apple"与"oranege"同属水果. 而且one-hot由于用一个向量表示整个词典, 造成向量非常大      
( 2 ) featurized representation是把单词用一系列属性表示, 用属性代替词典, 可以减小单词向量的维度. 同时可以给出更多关联信息.

#### 2. 类比(Analogy)词向量
使用词向量的余弦夹角来判断词向量的相似度. eg: ${ e }_{ man }-{ e }_{ women }={ e }_{ king }-{ e }_{ queen }$

#### 3. embedding-matrix潜入矩阵
当我们学习一个Word-Embedding时, 实际上是学习一个嵌入式矩阵.